# NLCT (11-3.9um BTD) visualized from Himawari

### Can visualize the daily NLCT, or the longterm average. Data is accessed from Amazon Web Solutions.
* Dan Lindsey's presentation on AHI data on AWS: https://docs.opendata.aws/noaa-himawari/2020July08_NOAA_Himawari.pdf

In [1]:
#---Cloud search libraries
import s3fs
import requests
import fnmatch

#---Data libraries
import xarray as xr
import netCDF4
import numpy as np
import datetime

#---Plotting libraries
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature

**Hooking up the AWS S3 buckets:**

In [2]:
fs = s3fs.S3FileSystem(anon=True)

**Set the datetime range of interest:**

In [3]:
year = 2024
month = 2
month_name = 'Feb'
day_start = 14
day_end = 14
hour = 7

**Getting Himawari products:**

In [4]:
products = fs.ls('noaa-himawari9')

In [5]:
products

['noaa-himawari9/AHI-L1b-FLDK',
 'noaa-himawari9/AHI-L1b-Japan',
 'noaa-himawari9/AHI-L1b-Target',
 'noaa-himawari9/AHI-L2-FLDK-Clouds',
 'noaa-himawari9/AHI-L2-FLDK-ISatSS',
 'noaa-himawari9/AHI-L2-FLDK-Winds',
 'noaa-himawari9/index.html']

**Getting Himawari radiance data:**

In [6]:
bucket = 'noaa-himawari9'
product = 'AHI-L1b-FLDK'

**Gathering a month of top-of-the-hour data from ABI band 07 and band 14:**
* currently grappling S0110, from a range of S0210, S0310, S0410 ... S1010

In [7]:
range(day_end-(day_start-1))

range(0, 1)

In [8]:
b07_data = []
b14_data = []
day = day_start

for i in range(day_end-(day_start-1)):
    julian = datetime.datetime(year, month, day).strftime('%j')
    data_path = bucket + '/' + product + '/'  + str(year) + '/' + str(month).zfill(2) + '/' + str(day) + '/' + str(hour).zfill(2) + '00'
    files = fs.ls(data_path)

    files_b07 = [file for file in files if fnmatch.fnmatchcase(file.split('/')[-1], "*B07_FLDK_R20_S0110.DAT.bz2")]
    files_b14 = [file for file in files if fnmatch.fnmatchcase(file.split('/')[-1], "*B14_FLDK_R20_S0110.DAT.bz2")]

    #---index of 0 to get top-of-the-hour
    b07_data.append(files_b07[0])
    b14_data.append(files_b14[0])

    day = day+1

In [9]:
b14_data

['noaa-himawari9/AHI-L1b-FLDK/2024/02/14/0700/HS_H09_20240214_0700_B14_FLDK_R20_S0110.DAT.bz2']

**Functions for calculating geostationary lat/lon:**

In [10]:
def calc_latlon(ds):
    # The math for this function was taken from 
    # https://makersportal.com/blog/2018/11/25/goes-r-satellite-latitude-and-longitude-grid-projection-algorithm
    x = ds.x
    y = ds.y
    goes_imager_projection = ds.goes_imager_projection
    
    x,y = np.meshgrid(x,y)
    
    r_eq = goes_imager_projection.attrs["semi_major_axis"]
    r_pol = goes_imager_projection.attrs["semi_minor_axis"]
    l_0 = goes_imager_projection.attrs["longitude_of_projection_origin"] * (np.pi/180)
    h_sat = goes_imager_projection.attrs["perspective_point_height"]
    H = r_eq + h_sat
    
    a = np.sin(x)**2 + (np.cos(x)**2 * (np.cos(y)**2 + (r_eq**2 / r_pol**2) * np.sin(y)**2))
    b = -2 * H * np.cos(x) * np.cos(y)
    c = H**2 - r_eq**2
    
    #--- Added absolute to remove error
    r_s = (-b - np.sqrt(np.absolute(b**2 - 4*a*c)))/(2*a)
    
    s_x = r_s * np.cos(x) * np.cos(y)
    s_y = -r_s * np.sin(x)
    s_z = r_s * np.cos(x) * np.sin(y)
    
    lat = np.arctan((r_eq**2 / r_pol**2) * (s_z / np.sqrt((H-s_x)**2 +s_y**2))) * (180/np.pi)
    lon = (l_0 - np.arctan(s_y / (H-s_x))) * (180/np.pi)
    
    ds = ds.assign_coords({
        "lat":(["y","x"],lat),
        "lon":(["y","x"],lon)
    })
    ds.lat.attrs["units"] = "degrees_north"
    ds.lon.attrs["units"] = "degrees_east"
    return ds



def get_xy_from_latlon(ds, lats, lons):
    lat1, lat2 = lats
    lon1, lon2 = lons

    lat = ds.lat.data
    lon = ds.lon.data
    
    x = ds.x.data
    y = ds.y.data
    
    x,y = np.meshgrid(x,y)
    
    x = x[(lat >= lat1) & (lat <= lat2) & (lon >= lon1) & (lon <= lon2)]
    y = y[(lat >= lat1) & (lat <= lat2) & (lon >= lon1) & (lon <= lon2)] 
    
    return ((min(x), max(x)), (min(y), max(y)))

**Setting the lat/lon range of the imagery:**

In [11]:
#--- Kuroshio
min_lon = 125
min_lat = 30
max_lon = 46
max_lat = 154

lats = (min_lat, max_lat)
lons = (min_lon, max_lon)

**Function for creating the 11um - 3.9um BTD:**

In [12]:
def create_BTD(data_07, data_14, filename):

    year = filename[27:31]
    jul_day = filename[31:34]
    h = filename[34:36]
    m = filename[36:38]

    print('Processing 07 band for '+year+'-'+jul_day+' '+h+':'+m)

    ds_07 = calc_latlon(data_07)

    ((x1,x2), (y1, y2)) = get_xy_from_latlon(ds_07, lats, lons)

    subset_07 = ds_07.sel(x=slice(x1, x2), y=slice(y2, y1))

    print('Processing 14 band for '+year+'-'+jul_day+' '+h+':'+m)

    ds_14 = calc_latlon(data_14)

    ((x1,x2), (y1, y2)) = get_xy_from_latlon(ds_14, lats, lons)

    subset_14 = ds_14.sel(x=slice(x1, x2), y=slice(y2, y1))

    #--- Calculate BTD and take product of the data over time
    T_07 = (subset_07.planck_fk2/(np.log((subset_07.planck_fk1/subset_07.Rad)+1)) - subset_07.planck_bc1)/subset_07.planck_bc2
    T_14 = (subset_14.planck_fk2/(np.log((subset_14.planck_fk1/subset_14.Rad)+1)) - subset_14.planck_bc1)/subset_14.planck_bc2

    BTD = T_14 - T_07
    
    yr_m_d = datetime.datetime.strptime(year+jul_day, '%Y%j').date()
    time_delta = datetime.timedelta(hours=int(h), minutes=int(m))
    dt = datetime.datetime.combine(yr_m_d, datetime.datetime.min.time()) + time_delta
    BTD = BTD.expand_dims({'time':[dt]})
    
    return BTD

**Single day NLCT: Choose date from list, accessing from AWS, then processing to create the BTD:**

* Stuck here. The Himawari data is in a compressed bz2 format, which uncompressed to some sort of binary.
* There is code available to convert Himawari data to netcdf (https://www.data.jma.go.jp/mscweb/en/himawari89/space_segment/spsg_sample.html)

In [13]:
import bz2

In [14]:
resp = requests.get(f'https://'+bucket+'.s3.amazonaws.com/'+b07_data[0][15:])

nc_07 = bz2.decompress(resp.content)

In [24]:
date_index = 0

resp = requests.get(f'https://'+bucket+'.s3.amazonaws.com/'+b07_data[date_index][15:])
if str(resp) != '<Response [200]>':
    print('b07 file not found in AWS servers')

nc_07 = netCDF4.Dataset(b07_data[date_index], memory = resp.content)
ds_07 = xr.open_dataset(xr.backends.NetCDF4DataStore(nc_07))

resp = requests.get(f'https://'+bucket+'.s3.amazonaws.com/'+b14_data[date_index][15:])
if str(resp) != '<Response [200]>':
    print('b14 file not found in AWS servers')

nc_14 = netCDF4.Dataset(b14_data[date_index], memory = resp.content)
ds_14 = xr.open_dataset(xr.backends.NetCDF4DataStore(nc_14))

filename = b07_data[date_index].split('/')[-1]

TypeError: filename must be a str, bytes, file or PathLike object

In [ ]:
BTD = create_BTD(ds_07, ds_14, filename)

In [ ]:
dt = BTD.time.values[0]
date_str = np.datetime_as_string(dt)[:10]
time_str = np.datetime_as_string(dt)[11:16]


projection=ccrs.PlateCarree()
fig,ax=plt.subplots(1, figsize=(12,12),subplot_kw={'projection': projection})
cmap = plt.cm.Spectral_r
levels = np.linspace(0, np.max(BTD), 21)

c=ax.contourf(BTD.lon, BTD.lat, BTD[0], cmap=cmap, extend='both', levels=levels)
clb=plt.colorbar(c, shrink=0.3, pad=0.02, ax=ax)
ax.set_title('11um - 3.9um BTD ('+date_str +' '+time_str+' UTC)', fontsize=18)
clb.set_label('BTD (K)')

ax.add_feature(cfeature.STATES)
ax.add_feature(cfeature.BORDERS)
ax.add_feature(cfeature.COASTLINE)

#plt.savefig("sample_images/nlct_"+date_str[0:4]+date_str[5:7]+date_str[8:10]+"_"+time_str[0:2], bbox_inches='tight')

**Average NLCT: Run over the time selected, accessing from AWS, the processing to create the BTD:**

In [ ]:
BTD_ds = []

for i in range(len(b07_data)):
    resp = requests.get(f'https://'+bucket+'.s3.amazonaws.com/'+b07_data[i][12:])
    if str(resp) != '<Response [200]>':
            print('b07 file not found in AWS servers')

    resp = requests.get(f'https://'+bucket+'.s3.amazonaws.com/'+b14_data[i][12:])
    if str(resp) != '<Response [200]>':
            print('b14 file not found in AWS servers')
    
    nc_07 = netCDF4.Dataset(b07_data[i], memory = resp.content)
    ds_07 = xr.open_dataset(xr.backends.NetCDF4DataStore(nc_07))

    nc_14 = netCDF4.Dataset(b14_data[i], memory = resp.content)
    ds_14 = xr.open_dataset(xr.backends.NetCDF4DataStore(nc_14))

    filename = b07_data[i].split('/')[-1]

    BTD = create_BTD(ds_07, ds_14, filename)

    BTD_ds.append(BTD)

In [ ]:
nlct_ds = xr.concat(BTD_ds, dim='time')

In [ ]:
nlct_ds

**Calculating the average BTD over the time range:**

In [ ]:
count = len(nlct_ds.time)
btd_tot = nlct_ds.sum(dim='time')
btd_mean = btd_tot/count

In [ ]:
overall_mean = btd_mean.mean(dim='y')
overall_mean = overall_mean.mean(dim='x')

In [ ]:
high_lim = overall_mean.values
low_lim = overall_mean.values

In [ ]:
projection=ccrs.PlateCarree()
fig,ax=plt.subplots(1, figsize=(12,12),subplot_kw={'projection': projection})
cmap = plt.cm.Spectral_r
levels = np.linspace(0, 1, 21)

c=ax.contourf(btd_mean.lon, btd_mean.lat, btd_mean, cmap=cmap, extend='both', levels=levels)
clb=plt.colorbar(c, shrink=0.3, pad=0.02, ax=ax)
ax.set_title('Average '+month_name+' NLCT Positive BTD (2022-2023)', fontsize=24)
clb.set_label('BTD (K)')

ax.add_feature(cfeature.STATES)
ax.add_feature(cfeature.BORDERS)
ax.add_feature(cfeature.COASTLINE)